In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./mnist/data/', one_hot = True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [13]:
learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_inputs = 28
n_steps = 28
n_hidden = 128
n_class = 10



- 코드 작성

In [4]:
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
Y=tf.placeholder(tf.float32, [None, n_class])

In [7]:
cell = tf.contrib.rnn.BasicRNNCell(n_hidden)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

#states = tf.zeros(batch_size)
#for i in range(n_steps):
#    outputs, states = cell(X[[:, i]], states)

- RNN에서 나온 출력값을 최종 출력값을 만들기 위해서는 최종결과값이 실측값 Y와 동일한 형태인 [batch_size, n_class]가 되어야한다.

In [10]:
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

#outputs : [batch_size, n_step, n_hidden]
# ----> [n_step, batch_size, n_hidden]

outputs = tf.transpose(outputs, [1, 0, 2])
#-> [batch_size, n_hidden]
outputs = outputs[-1]

In [11]:
model = tf.matmul(outputs, W) + b

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = model))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy =tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/ batch_size)

test_batch_size = len(mnist.test.images)
for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(batch_size, n_steps, n_inputs)
        
        _, cost_val= sess.run([train, cost], feed_dict = {X: batch_xs, Y: batch_ys})
        
        total_cost += cost_val
    
    acc = sess.run(accuracy, feed_dict = {X: mnist.test.images.reshape(test_batch_size, n_steps, n_inputs), Y: mnist.test.labels})
    print('Epoch : {}, avg_cost : {}, accuracy : {}'.format(epoch, cost_val, acc))

Epoch : 0, avg_cost : 0.36812928318977356, accuracy : 0.9177001118659973
Epoch : 1, avg_cost : 0.35402435064315796, accuracy : 0.9417001008987427
Epoch : 2, avg_cost : 0.19710153341293335, accuracy : 0.9523001313209534
Epoch : 3, avg_cost : 0.05755286291241646, accuracy : 0.955000102519989
Epoch : 4, avg_cost : 0.11476858705282211, accuracy : 0.9647001624107361
Epoch : 5, avg_cost : 0.2467179149389267, accuracy : 0.947900116443634
Epoch : 6, avg_cost : 0.17295950651168823, accuracy : 0.9614001512527466
Epoch : 7, avg_cost : 0.0734465941786766, accuracy : 0.9675001502037048
Epoch : 8, avg_cost : 0.08358713984489441, accuracy : 0.9724001288414001
Epoch : 9, avg_cost : 0.09936118125915527, accuracy : 0.9653000831604004
Epoch : 10, avg_cost : 0.20417945086956024, accuracy : 0.9618001580238342
Epoch : 11, avg_cost : 0.1080816388130188, accuracy : 0.9745001792907715
Epoch : 12, avg_cost : 0.10721761733293533, accuracy : 0.9721001386642456
Epoch : 13, avg_cost : 0.22235408425331116, accuracy 